In [ ]:
import jieba
import os
import pickle
import pandas as pd
import re
import copy
import numpy as np
from functools import reduce
from datetime import datetime, timedelta
from sklearn.cluster import KMeans, MiniBatchKMeans,DBSCAN
from pymongo import MongoClient
import pprint
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import jieba.posseg as pseg
import gensim
from gensim import corpora,models
import concurrent
from concurrent.futures import ThreadPoolExecutor

In [2]:
with open("Events_eps0.7_5days","rb") as fp:
    Events = pickle.load(fp)
Events = Events.reset_index(drop=True)
# Check result after process in this file
Events.head(3)

,_id,diff,dmax,dmin,org_indexes,subtag,tag,hundred_billion,hundred_million,ten_million,...,topic_30,topic_31,topic_32,topic_33,topic_34,topic_35,topic_36,topic_37,topic_38,topic_39
0,"{'tag': 0, 'subtag': 2}",29.0,2017-03-15,2017-02-14,"[20811, 20802, 20779, 18617, 701, 723, 641, 20...",2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"{'tag': 34, 'subtag': 3}",24.0,2017-07-15,2017-06-21,"[21170, 21157, 1362, 1439, 1361, 1321, 19213, ...",3,34,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"{'tag': 34, 'subtag': 0}",21.0,2017-05-18,2017-04-27,"[1101, 1054, 1117, 18992, 1147, 18898, 1051, 1...",0,34,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# valid
with open("Valid_events_eps0.7_5days","rb") as fp:
    Events_v = pickle.load(fp)
Events_v = Events_v.reset_index(drop=True)
# Check result after process in this file
Events_v.head(3)

,_id,diff,dmax,dmin,org_indexes,subtag,tag,hundred_billion,hundred_million,ten_million,...,topic_30,topic_31,topic_32,topic_33,topic_34,topic_35,topic_36,topic_37,topic_38,topic_39
0,"{'tag': 0, 'subtag': 0}",5.0,2018-01-11,2018-01-06,"[204, 187, 180, 225, 159, 179, 167, 175]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"{'tag': 1, 'subtag': 0}",3.0,2018-01-08,2018-01-05,"[164, 163, 298, 157]",0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"{'tag': 2, 'subtag': 0}",0.0,2017-12-14,2017-12-14,"[297, 296, 295, 294, 293, 292, 291, 290, 289, ...",0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
with open("news_all_with_event","rb") as fp:
    Event_News_All = pickle.load(fp)
Event_News_All.iloc[0,:]

_id                                  5a4441b10139d5015cd72435
category                                             politics
content     北市勞動局限國民黨18日前給付員工9月薪資，但迄昨下班前未收到回覆；勞動局說，今會再確認欠薪...
datetime                                  2016-10-19 00:00:00
hash                                      5332400969091782388
id                                      20161019001278-260118
location                                                   台北
media                                              chinatimes
reporter                                          黃意涵、楊孟立、林思慧
title                                北市勞動局限18日前給薪 國民黨欠薪 最快今挨罰
url         http://www.chinatimes.com//newspapers/20161019...
deter                                                    True
Name: 0, dtype: object

In [5]:
# valid
with open("valid_news_all_with_event","rb") as fp:
    Event_News_All_v = pickle.load(fp)
Event_News_All_v.iloc[0,:]

_id                                  5a5cac486f7e9f553488371b
category                                             politics
content     國防部昨提五大違約事項函告慶富解約。不過藍委馬文君認為，國防部宣告違約的理由剛好證實軍方發言...
datetime                                           2017-12-14
hash                                     -6267535548327685137
id                                      20171214000461-260118
location                                                   台北
media                                              chinatimes
reporter                                               周毓翔、鄭舲
title                          藍委馬文君踢爆解約理由前後矛盾！軍方一路護航慶富 解約圖止血
url         http://www.chinatimes.com//newspapers/20171214...
deter                                                    True
Name: 0, dtype: object

In [ ]:
# 初始化產生一個NewFeatue的欄位
def FindMoney(Events):
    Events["hundred_billion"] = 0
    Events["hundred_million"] = 0
    Events["ten_million"] = 0
    Events["million"] = 0
    Events["hundred_thousand"] = 0
    for j in range(len(Events["hundred_billion"])):
        for i in (Events["org_indexes"][j]):
        # 以下取得每一則新聞的Datagrame
            if re.match(".*兆元.*",Event_News_All.iloc[i,2]):
                Events.loc[j,"hundred_billion"] = 1
                break
            elif re.match(".*億元.*",Event_News_All.iloc[i,2]):
                Events.loc[j,"hundred_million"] = 1
                break
            elif re.match(".*千萬元.*",Event_News_All.iloc[i,2]):
                Events.loc[j,"ten_million"] = 1
                break
            elif re.match(".*百萬元.*",Event_News_All.iloc[i,2]):
                Events.loc[j,"million"] = 1
                break
            elif re.match(".*十萬元.*",Event_News_All.iloc[i,2]):
                Events.loc[j,"hundred_thousand"] = 1
                break
        print("findMoney"+str(j)+"cluster")
FindMoney(Events)
FindMoney(Events_v)

In [28]:
def AddPolitician(Events):
    Names=["姚文智","蔡正元","侯友宜","盧秀燕","江啟臣","陳其邁","鄭文燦","蔡英文","柯文哲","黃國昌","馬英九","賴清德","洪慈庸","朱立倫","林昶佐","陳菊","李錫錕","丁守中","林佳龍","蔣萬安"]
    for n in Names:
        Events[n] = 0
        regex = ".*"+re.escape(n)+".*"
        for j in range(len(Events[n])):
            for i in (Events["org_indexes"][j]):
                # 以下取得每一則新聞的Datagrame
                if re.match(regex,Event_News_All.iloc[i,2]):
                    Events.loc[j,n]=1
                    print("AddPolitician"+str(n)+str(j)+"cluster")
                    break
                
                    
AddPolitician()

AddPolitician姚文智21cluster
AddPolitician姚文智38cluster
AddPolitician姚文智39cluster
AddPolitician姚文智55cluster
AddPolitician姚文智63cluster
AddPolitician姚文智81cluster
AddPolitician姚文智93cluster
AddPolitician蔡正元27cluster
AddPolitician蔡正元184cluster
AddPolitician侯友宜87cluster
AddPolitician侯友宜193cluster
AddPolitician侯友宜214cluster
AddPolitician盧秀燕214cluster
AddPolitician江啟臣26cluster
AddPolitician江啟臣169cluster
AddPolitician江啟臣190cluster
AddPolitician江啟臣214cluster
AddPolitician鄭文燦66cluster
AddPolitician蔡英文0cluster
AddPolitician蔡英文1cluster
AddPolitician蔡英文5cluster
AddPolitician蔡英文7cluster
AddPolitician蔡英文16cluster
AddPolitician蔡英文18cluster
AddPolitician蔡英文20cluster
AddPolitician蔡英文31cluster
AddPolitician蔡英文38cluster
AddPolitician蔡英文52cluster
AddPolitician蔡英文56cluster
AddPolitician蔡英文66cluster
AddPolitician蔡英文69cluster
AddPolitician蔡英文77cluster
AddPolitician蔡英文79cluster
AddPolitician蔡英文81cluster
AddPolitician蔡英文82cluster
AddPolitician蔡英文84cluster
AddPolitician蔡英文87cluster
AddPolitician蔡英文88cluster
AddPoliti

In [29]:
def EventDensity(Events):
    Events["EventDensity"] = 0
    for j in range(len(Events["EventDensity"])):
        drange = timedelta(days=2)
        #a ball around dmax         
        upperbound = Events.loc[j,"dmax"]+drange
        lowerbound = Events.loc[j,"dmax"]-drange
        #Because it will always count itself.  
        count = -1
        for  k in range(len(Events["EventDensity"])):
            if lowerbound<= Events.loc[k,"dmax"] <= upperbound:
                count+=1
            elif lowerbound<= Events.loc[k,"dmin"] <= upperbound:
                count+=1
        Events.loc[j,"EventDensity"] = count
        print("finish"+str(j)+"cluster")
EventDensity()
    

finish0cluster
finish1cluster
finish2cluster
finish3cluster
finish4cluster
finish5cluster
finish6cluster
finish7cluster
finish8cluster
finish9cluster
finish10cluster
finish11cluster
finish12cluster
finish13cluster
finish14cluster
finish15cluster
finish16cluster
finish17cluster
finish18cluster
finish19cluster
finish20cluster
finish21cluster
finish22cluster
finish23cluster
finish24cluster
finish25cluster
finish26cluster
finish27cluster
finish28cluster
finish29cluster
finish30cluster
finish31cluster
finish32cluster
finish33cluster
finish34cluster
finish35cluster
finish36cluster
finish37cluster
finish38cluster
finish39cluster
finish40cluster
finish41cluster
finish42cluster
finish43cluster
finish44cluster
finish45cluster
finish46cluster
finish47cluster
finish48cluster
finish49cluster
finish50cluster
finish51cluster
finish52cluster
finish53cluster
finish54cluster
finish55cluster
finish56cluster
finish57cluster
finish58cluster
finish59cluster
finish60cluster
finish61cluster
finish62cluster
fi

In [30]:
# BigEvents = Events[(Events.diff != 0)]
# news_df = Events[(Events["diff"] != 1)]



In [31]:
def BigEventpeak(Events):
    Events["BigEventPeak"] = 0
    BigEvents = Events[(Events["diff"] != 0)]
    for j in range(len(Events["BigEventPeak"])):
        drange = timedelta(days=2)
        #a ball around dmax         
        upperbound = Events.loc[j,"dmax"]+drange
        lowerbound = Events.loc[j,"dmax"]-drange
        #Because it will always count itself.  
        count = -Events.loc[j,"diff"]
        for  k in range(len(BigEvents["BigEventPeak"])):
            if lowerbound<= BigEvents.loc[k,"dmax"] <= upperbound:
                count += BigEvents.loc[k,"diff"]
            elif lowerbound<= BigEvents.loc[k,"dmin"] <= upperbound:
                count += BigEvents.loc[k,"diff"]
        Events.loc[j,"BigEventPeak"] = count
        print("finish"+str(j)+"cluster")
        
BigEventpeak()

finish0cluster
finish1cluster
finish2cluster
finish3cluster
finish4cluster
finish5cluster
finish6cluster
finish7cluster
finish8cluster
finish9cluster
finish10cluster
finish11cluster
finish12cluster
finish13cluster
finish14cluster
finish15cluster
finish16cluster
finish17cluster
finish18cluster
finish19cluster
finish20cluster
finish21cluster
finish22cluster
finish23cluster
finish24cluster
finish25cluster
finish26cluster
finish27cluster
finish28cluster
finish29cluster
finish30cluster
finish31cluster
finish32cluster
finish33cluster
finish34cluster
finish35cluster
finish36cluster
finish37cluster
finish38cluster
finish39cluster
finish40cluster
finish41cluster
finish42cluster
finish43cluster
finish44cluster
finish45cluster
finish46cluster
finish47cluster
finish48cluster
finish49cluster
finish50cluster
finish51cluster
finish52cluster
finish53cluster
finish54cluster
finish55cluster
finish56cluster
finish57cluster
finish58cluster
finish59cluster
finish60cluster
finish61cluster
finish62cluster
fi

In [6]:
def topic_appending(Events):
    with open('news_all_important_noun',"rb") as f:
        news_important_noun = pickle.load(f)
    with open("LDA_model_news_all_with_event","rb")as fp:
        ldamodel = pickle.load(fp)
    texts = []
    for i in news_important_noun:
        texts.append(i.split(" "))
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    num_topics = 40
    ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=num_topics,id2word = dictionary,passes = 1)
    doc_topics = []
    for i in texts:
        bow = dictionary.doc2bow(i)
        doc_topics.append(ldamodel.get_document_topics(bow)[0])
    
    
    for n in range(num_topics):
        name ="topic_"+str(n)
        Events[name] = 0
        
    for j in range(len(Events["org_indexes"])):
        topics = []
        for i in (Events["org_indexes"][j]):
            topics.append(doc_topics[i][0])
        cluster_topics = np.array(topics)
        counts = np.bincount(cluster_topics)
        mode=np.argmax(counts)
        insert_column_name = "topic_"+str(mode)
        Events.loc[j,insert_column_name] = 1
        print("finish"+str(j)+"cluster")
        
    
topic_appending()


# checking!
# sumnum = 0
# for n in range(40):
#     name ="topic_"+str(n)
#     sumnum+=Events[name].sum()
# sumnum

In [20]:
def AddWords(Events):
#     with open('news_all_important_noun',"rb") as f:
#         news_important_noun = pickle.load(f)
#     allwords = []
#     for i in news_important_noun:
#         allwords.extend(i.split(" "))
#     allwords = set(allwords)
#     for n in allwords:
#         Events[n] = 0
#     print("preparation finished")
    
    for j in range(len(Events["diff"])):
    # for match
        temp = ""
        for i in (Events["org_indexes"][j]):
            temp += Event_News_All.iloc[i,2]
        print(str(j)+ " cluster news collected ")

        for n in allwords:
            Events.loc[j,n]+=len(re.findall(n, temp))

    #         regex = re.escape(n)
    #         if re.match(regex,temp):
    #             Events.loc[j,n]=1
        print("Addword"+" "+str(j)+" cluster")
    

#     for j in range(20806):
#         for i in contents[j]:
#             for n in allwords:
#                 Events.loc[j,n]+=len(re.findall(n, i))
#         print("Addword"+" "+str(j)+" cluster")
        
AddWords(Events)


preparation finished
0 cluster news collected 
Addword 0 cluster
1 cluster news collected 
Addword 1 cluster
2 cluster news collected 
Addword 2 cluster
3 cluster news collected 
Addword 3 cluster
4 cluster news collected 
Addword 4 cluster
5 cluster news collected 
Addword 5 cluster
6 cluster news collected 
Addword 6 cluster
7 cluster news collected 
Addword 7 cluster
8 cluster news collected 


KeyboardInterrupt: 

In [ ]:
#     for j in range(len(Events["diff"])):
#         # for match
# #         temp = ""
# #         for i in (Events["org_indexes"][j]):
# #             temp += Event_News_All.iloc[i,2]
# #         print(str(j)+ " cluster news collected ")
# #         for n in allwords:
# #             regex = ".*"+re.escape(n)+".*"

# #             if re.match(regex,temp):
# #                 Events.loc[j,n]=1
# #         print("Addword"+" "+str(j)+" cluster")
        
        
#         # for Count
#         temp = []
#         for i in (Events["org_indexes"][j]):
#             temp.extend(Event_News_All.iloc[i,2])
#         for n in allwords:
#             regex = re.compile(".*"+re.escape(n)+".*")
#             wordfilter = filter(regex.match, temp)
#             Events.loc[j,n]=len(list(wordfilter))
#         print("Addword"+" "+str(j)+" cluster")

        
#     contents = []
#     for j in range(len(Events["diff"])):
#         temp = []
#         for i in (Events["org_indexes"][j]):
#             temp.append(Event_News_All.iloc[i,2])
#         contents.append(temp)
#     print("finish contents")
    
#     for j in range(len(Events["diff"])):
#         for n in allwords:
#             for i in contents[j]:
#                 Events.loc[j,n]+=len(re.findall(n, i))
#         print("Addword"+" "+str(j)+" cluster")


In [6]:
with open('news_all_important_noun',"rb") as f:
        news_important_noun = pickle.load(f)
allwords = []
for i in news_important_noun:
    allwords.extend(i.split(" "))
allwords = set(allwords)
for n in allwords:
    Events[n] = 0
print("preparation finished")

contents = []
for j in range(len(Events["diff"])):
    temp = []
    for i in (Events["org_indexes"][j]):
        temp.append(Event_News_All.iloc[i,2])
    contents.append(temp)
print("finish contents")

preparation finished
finish contents


In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=200) as executor:
    for data in executor.map(AddWords, Events):
        print("runing_cluster")

In [16]:
contents[0]

['監察委員被提名人、總統府前秘書長陳師孟昨出席綠色逗陣座談會，談監察院存廢。陳說，監察院、考試院都應該廢掉，但監察院廢除前，可以廢物利用，當司法體制失控時，監察權還能亡羊補牢。他說，黨產會在行政法院頻頻受挫，就是因為司法仍在當國民黨打手。針對前總統陳水扁案、前交通部長郭瑤琪案，陳師孟昨被問若未來當上監委，是否會進行調查？陳師孟說，現在談這些言之過早，但若遇到志同道合的監委，會一起努力，做一些還給台灣人民公道的事。關於考試院、監察院的存廢，陳師孟說，當初他也認為考試、監察兩院趕快廢掉，但近來他感覺到，監察院應該廢物利用，他把矛頭指向司法人員說，經過國民黨長久洗腦教育，獨立辦案反而變成意識型態的包裝紙，明明有意識型態、黨國想法，卻說是自由心證，還大聲地說是根據自由心證，陳水扁就是要關、就是貪汙。針對近來黨產會查黨產，卻在行政法院頻頻受挫，陳師孟也說，這就是典型司法仍在當國民黨打手，不讓社會轉型正義的防線，儘管政黨輪替，但還是要幫忙護航黨產，因此監察權有必要在司法不公時，為司法亡羊補牢。他指出，未來做好這三件事，監察權就能完全廢除，第一就是除垢法，讓恐龍法官、迫害綠營政務官等得到應有懲罰；第二，陪審制代替法官所主導的制度，用陪審員的同情心、同理心，代替法官的自由心證。陳師孟說，如果法官產出還是由政府篩選，容易把政黨立場、意識型態灌輸在裡面，司法體制仍被政治、威權操控，因此第三點，他認為法官要民選，若做到這三點，把司法體制變成健康、正常體制，監察權就可以廢掉。',
 '發起連署將張靜移送懲戒的桃園地院法官孫健智指出，放縱律師謠傳司法官收賄，明天可能就換律師被有心人士謠傳，是靠著送錢才勝訴。長居台東的張靜昨天一早北上開記者會指出，不能說他稱有法官與檢察官貪汙，就等於「詆毀全世界的法官和檢察官」，每個人的經驗不同，不一樣的經驗說出不一樣的話很正常，看守所的被告最清楚，法界應有被批評的雅量。張靜說，不會說出受賄名單，但若遭懲戒而要到法庭為自己辯護時就會提出。張說，力推陪審制，這樣人們無從行賄，可以降低賄賂、不公平審判的機會。他強調，凡有人類就會有犯罪，陪審制並非就無缺點，也不保證陪審員不會貪汙。檢察官轉任律師的鄭嘉欣表示，民眾期待司法改革，但不該是讓司法崩解，她舉例，曾有被告在法庭上罵得臉紅脖子粗，法官卻心平氣和地從一大疊卷宗找出證據，其實「很多法官都認真地發掘真實」。',
 

In [10]:
len(Events["diff"])

20806

0
1
2
3
4


20806

In [33]:
set(Events['topic_39'])

{0}

In [30]:
import pprint
pprint.pprint(ldamodel.print_topics(num_topics=40,num_words=40))

NameError: name 'ldamodel' is not defined

In [ ]:
feature_cols = []

In [98]:
# Events = Events.drop('FindMoney', 1)

In [23]:
with open("Events_eps0.7_5days","wb") as fp:
#     Events = pickle.load(fp)
    pickle.dump(Events,fp)

In [35]:
# valid
with open("Valid_events_eps0.7_5days","wb") as fp:
#     Events = pickle.load(fp)
    pickle.dump(Events,fp)

In [27]:
Events.head(3)

,_id,diff,dmax,dmin,org_indexes,subtag,tag,hundred_billion,hundred_million,ten_million,...,供需,腳會,抗告,外地,主源,綠監,期待值,效法,法界,心道
0,"{'tag': 0, 'subtag': 2}",29.0,2017-03-15,2017-02-14,"[20811, 20802, 20779, 18617, 701, 723, 641, 20...",2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"{'tag': 34, 'subtag': 3}",24.0,2017-07-15,2017-06-21,"[21170, 21157, 1362, 1439, 1361, 1321, 19213, ...",3,34,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"{'tag': 34, 'subtag': 0}",21.0,2017-05-18,2017-04-27,"[1101, 1054, 1117, 18992, 1147, 18898, 1051, 1...",0,34,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
with open('news_all_important_noun',"rb") as f:
        news_important_noun = pickle.load(f)
with open("LDA_model_news_all_with_event","rb")as fp:
    ldamodel = pickle.load(fp)
texts = []
for i in news_important_noun:
    texts.append(i.split(" "))
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
num_topics = 40
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=num_topics,id2word = dictionary,passes = 1)

In [18]:
print(news_important_noun[0])
print(news_important_noun[1])

依法 勞局 程序 行政 程序 理由 程序 限期 公司 薪水 依法 部分 薪水 法定 薪水
作法 人力 老化 退休金 教育部 退休金 可能性 人力 老化 公司 基金 月薪 薪水 月薪 隔代


[(0,
  '0.116*"大家" + 0.038*"朋友" + 0.028*"站台" + 0.026*"夫人" + 0.025*"妻子" + 0.024*"官邸" '
  '+ 0.017*"全程" + 0.016*"主持人" + 0.015*"生涯" + 0.015*"小朋友" + 0.014*"心情" + '
  '0.013*"白色" + 0.013*"合影" + 0.012*"家人" + 0.012*"幕僚" + 0.011*"市政" + 0.011*"姊妹" '
  '+ 0.010*"接棒" + 0.010*"小孩" + 0.010*"行程" + 0.009*"力量" + 0.009*"感情" + '
  '0.009*"全場" + 0.009*"全台" + 0.009*"舞台" + 0.009*"外界" + 0.009*"老家" + 0.008*"家庭" '
  '+ 0.008*"衣服" + 0.008*"丈夫" + 0.008*"妹妹" + 0.007*"歌手" + 0.007*"作家" + '
  '0.007*"理念" + 0.007*"原本" + 0.007*"居家" + 0.007*"感性" + 0.006*"笑容" + 0.006*"精彩" '
  '+ 0.006*"代表"'),
 (1,
  '0.085*"政治" + 0.043*"人物" + 0.041*"爆料" + 0.034*"形象" + 0.034*"人士" + 0.023*"事情" '
  '+ 0.020*"前台" + 0.019*"外界" + 0.018*"爸爸" + 0.016*"受刑人" + 0.014*"照片" + '
  '0.013*"故事" + 0.012*"身分" + 0.012*"無人" + 0.011*"朋友" + 0.011*"主席" + 0.011*"信仰" '
  '+ 0.010*"人心" + 0.010*"大家" + 0.010*"服刑" + 0.010*"司令部" + 0.010*"消息" + '
  '0.010*"名嘴" + 0.010*"父子" + 0.009*"司法" + 0.008*"地方" + 0.008*"行事" + 0.008*"私下" '
  '+ 0.007*"友人" + 0.007*"感情" + 0.007*"召集